## Load documents

In [1]:
import glob
import json
import os
from tqdm import tqdm
from pathlib import Path
from llama_index.core import (
    SimpleDirectoryReader,
    Document,
    Settings,
    VectorStoreIndex,
    StorageContext,
    SummaryIndex
)
from llama_index.readers.file import HTMLTagReader, FlatReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import HTMLNodeParser, MarkdownNodeParser, SentenceSplitter
from llama_index.core.storage.docstore import SimpleDocumentStore
from IPython.display import display, HTML, Markdown

/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs_metadata = json.load(open("./data/llama_blogs_metadata.json", "r"))
len(docs_metadata)

166

### Load html documents

In [2]:
html_loader = SimpleDirectoryReader(
    input_dir="./data/llama-blogs-html/",
    filename_as_id=True,
)

In [3]:
html_documents = html_loader.load_data(show_progress=True, num_workers=8)

In [4]:
test_doc = html_documents[10]

In [5]:
test_doc.metadata

{'file_path': '/workspace/projects/LlamindexHelper/data/llama-blogs-html/bridging-the-gap-in-crisis-counseling-introducing-counselor-copilot-db42e26ab4f3.html',
 'file_name': 'bridging-the-gap-in-crisis-counseling-introducing-counselor-copilot-db42e26ab4f3.html',
 'file_type': 'text/html',
 'file_size': 11333,
 'creation_date': '2024-08-08',
 'last_modified_date': '2024-08-08'}

In [6]:
display(HTML(test_doc.text))

### Preview markdown document loader

In [3]:
md_reader = FlatReader()
parser = MarkdownNodeParser()

In [27]:
test_doc_metadata = docs_metadata[50]
test_doc_metadata

{'title': 'Towards Long Context RAG',
 'date': 'Mar 1, 2024',
 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}

In [28]:
md_dir = "./data/llama-blogs-md/"

In [29]:
file_path = os.path.join(md_dir, test_doc_metadata["url"].split("/")[-1] + ".md")
file_path

'./data/llama-blogs-md/towards-long-context-rag.md'

In [15]:
md_docs = md_reader.load_data(Path(file_path), extra_info=doc0_metadata)

In [16]:
md_docs[0].metadata

{'filename': 'towards-long-context-rag.md',
 'extension': '.md',
 'title': 'Towards Long Context RAG',
 'date': 'Mar 1, 2024',
 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}

In [18]:
nodes = parser.get_nodes_from_documents(md_docs, show_progress=True)

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 400.49it/s]


In [19]:
len(nodes)

9

In [22]:
for node in nodes:
    print(node.metadata)

{'filename': 'towards-long-context-rag.md', 'extension': '.md', 'title': 'Towards Long Context RAG', 'date': 'Mar 1, 2024', 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}
{'Header_2': ' Our Mission Goes Beyond RAG', 'filename': 'towards-long-context-rag.md', 'extension': '.md', 'title': 'Towards Long Context RAG', 'date': 'Mar 1, 2024', 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}
{'Header_2': ' Initial Gemini 1.5 Pro Observations', 'filename': 'towards-long-context-rag.md', 'extension': '.md', 'title': 'Towards Long Context RAG', 'date': 'Mar 1, 2024', 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}
{'Header_2': ' Long Contexts Resolve Some Pain Points, but some Challenges Remain', 'filename': 'towards-long-context-rag.md', 'extension': '.md', 'title': 'Towards Long Context RAG', 'date': 'Mar 1, 2024', 'url': 'https://www.llamaindex.ai/blog/towards-long-context-rag'}
{'Header_2': ' Towards New RAG Architectures', 'filename': 't

### Markdown documents injestion

In [4]:
md_reader = FlatReader()
parser = MarkdownNodeParser()

In [5]:
md_dir = "./data/llama-blogs-md/"

In [6]:
loaded_documents = []
for doc_metadata in tqdm(docs_metadata, desc="Parsing documents"):
    file_path = os.path.join(md_dir, doc_metadata["url"].split("/")[-1] + ".md")
    md_docs = md_reader.load_data(Path(file_path), extra_info=doc_metadata)
    loaded_documents = loaded_documents + md_docs

Parsing documents: 100%|██████████| 166/166 [00:00<00:00, 11530.83it/s]


In [7]:
len(loaded_documents)

166

In [21]:
# store documents
md_docs_store = SimpleDocumentStore()

In [22]:
md_docs_store.add_documents(nodes)

In [25]:
md_docs_store.persist("database/llama-blogs-nodes-md-parser.json")

### Split documents to nodes

In [8]:
nodes = parser.get_nodes_from_documents(loaded_documents, show_progress=True)

Parsing nodes: 100%|██████████| 166/166 [00:00<00:00, 1001.43it/s]


In [9]:
len(nodes)

1221

In [11]:
for node in nodes[:10]:
    display(Markdown(node.text))
    print(node.metadata)
    print("=="*50)

Greetings, Llama Lovers!

Welcome to this week’s edition of the LlamaIndex newsletter! We’re excited to
share our latest updates including dynamic features like LlamaIndex Workflows
and retrieval capabilities in LlamaCloud. Check out our in-depth guides,
tutorials, and the upcoming webinars that will help you make the most of these
new developments.

{'filename': 'llamaindex-newsletter-2024-08-06.md', 'extension': '.md', 'title': 'LlamaIndex Newsletter 2024-08-06', 'date': 'Aug 6, 2024', 'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-08-06'}


**The highlights:**

  1. **LlamaIndex Workflows Launched:** LlamaIndex Workflows, a new event-driven architecture for building multi-agent applications, supports batching, async operations, and streaming. Agents subscribe to and emit events for complex, readable, Pythonic orchestration. [ Blogpost ](https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex) , [ Tweet ](https://x.com/llama_index/status/1819048068798616058) . 
  2. **Dynamic Retrieval Feature in LlamaCloud:** A new feature in LlamaCloud now supports dynamic retrieval for QA assistants, enabling both chunk-level and file-level document retrieval based on query similarity to intelligently route queries. [ Blogpost ](https://www.llamaindex.ai/blog/dynamic-retrieval-with-llamacloud) , [ Notebook ](https://github.com/run-llama/llamacloud-demo/blob/main/examples/10k_apple_tesla/demo_file_retrieval.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818337133746360623) . 
  3. **LongRAG LlamaPack:** LongRAG is now available as a LlamaPack in LlamaIndex, utilizing larger document chunks and long-context LLMs for more effective synthesis. [ Notebook ](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-longrag/examples/longrag.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818802688274100578) .

{'Header_2': ' **The highlights:**', 'filename': 'llamaindex-newsletter-2024-08-06.md', 'extension': '.md', 'title': 'LlamaIndex Newsletter 2024-08-06', 'date': 'Aug 6, 2024', 'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-08-06'}


**Feature Releases and Enhancements:**

  * We have launched LlamaIndex Workflows, a new event-driven way to build multi-agent applications where each agent acts as a component that subscribes to and emits events, allowing for complex, readable, and Pythonic orchestration with enhanced support for batching, async operations, and streaming. [ Blogpost ](https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex) , [ Tweet ](https://x.com/llama_index/status/1819048068798616058) . 
  * We have introduced a new feature in LlamaCloud to improve your QA assistant with our latest capability for dynamic retrieval, allowing both chunk-level and file-level retrieval. This feature enables the retrieval of entire documents based on query similarity, which supports building agents that can intelligently route queries based on their content. [ Blogpost ](https://www.llamaindex.ai/blog/dynamic-retrieval-with-llamacloud) , [ Notebook ](https://github.com/run-llama/llamacloud-demo/blob/main/examples/10k_apple_tesla/demo_file_retrieval.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818337133746360623) . 
  * We have launched LongRAG as a LlamaPack in LlamaIndex. LongRAG simplifies retrieval by using larger document chunks and leveraging long-context LLMs for synthesis. [ Notebook ](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-longrag/examples/longrag.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818802688274100578) . 

**Guides:**

  * [ Guide ](https://docs.llamaindex.ai/en/latest/examples/workflow/react_agent/) to building a ReAct agent from scratch using LlamaIndex workflows. 
  * [ Guide ](https://docs.llamaindex.ai/en/latest/examples/workflow/rag/) to Building an Event-Driven RAG Pipeline with LlamaIndex, featuring distinct event-driven steps for retrieval, reranking, and synthesis, enhanced with graph tracing and async processing. 
  * [ Guide ](https://docs.llamaindex.ai/en/latest/module_guides/observability/#mlflow) to MLflow in LlamaIndex to manage, deploy, and monitor your genAI applications with MLflow's tracking, packaging, evaluation, and tracing capabilities. 

**Tutorials:**

  * [ Pavan Kumar’s ](https://x.com/pavan_mantha1) [ tutorial ](https://blog.gopenai.com/building-smarter-agents-using-llamaindex-agents-and-qdrants-hybrid-search-50c0ecbbfb0d) on Building Smarter Agents using LlamaIndex Agents and Qdrant’s Hybrid Search. 
  * [ Farzad Sunavala’s ](https://www.linkedin.com/in/farzadsunavala) [ tutorial ](https://farzzy.hashnode.dev/rag-observability-and-evaluation-with-azure-ai-search-azure-openai-llamaindex-and-arize-phoenix) on RAG Observability and Evaluation with Azure AI Search, Azure OpenAI, LlamaIndex, and Arize Phoenix. 
  * [ Composio’s ](https://x.com/composiohq) [ tutorial ](https://github.com/ComposioHQ/composio/tree/master/python/examples/pr_agent/pr_agent_llama_index) on building a PR review agent using Composio's GitHub/Slack tools and LlamaIndex agent abstractions. 
  * [ Benito Martin’s ](https://medium.com/@benitomartin) [ tutorial ](https://medium.com/@benitomartin/find-your-code-scaling-a-llamaindex-and-qdrant-application-with-google-kubernetes-engine-2db126f16344) on Scaling a LlamaIndex and Qdrant Application with Google Kubernetes Engine. 
  * [ Chew Loong Nian’s ](https://medium.com/@chewloongnian) [ tutorial ](https://pub.towardsai.net/introducing-llamaextract-beta-transforming-metadata-extraction-for-enhanced-rag-queries-de3d74d34cd7) on Transforming Metadata Extraction for Enhanced RAG Queries using LlamaExtract. 
  * [ Pavan Kumar’s ](https://x.com/pavan_mantha1) [ tutorial ](https://medium.com/@manthapavankumar11/practical-implementation-of-agentic-rag-workflows-with-llama-index-and-qdrant-3b6622cd3124) on Practical Implementation of Agentic RAG Workflows with Llama-Index and Qdrant. 
  * AI21 Labs [ tutorial ](https://www.llamaindex.ai/blog/jamba-instruct-s-256k-context-window-on-llamaindex) on using Jamba-Instruct Model with LlamaIndex. 

**Webinars And Hackathons:**

  * [ Join us ](https://lu.ma/ka5xtyqo) for a webinar on August 8th with [ Dedy Kredo ](https://x.com/DedyKredo) from [ CodiumAI ](https://x.com/CodiumAI) on using RAG with LlamaIndex to help build a code generation solution that’s contextually aware of the right elements of source code. 
  * [ Join us ](https://lu.ma/p13pkknm?tk=SsniSt) on RAG Hack Night at GitHub with [ Weaviate ](https://x.com/weaviate_io) , [ Neosync ](https://x.com/neosynccloud) , [ Arize AI ](https://x.com/arizeai) on August 13th.

...

{'Header_2': ' A workflow primer', 'Header_3': ' Workflows can loop', 'filename': 'introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex.md', 'extension': '.md', 'title': 'Introducing workflows beta: a new way to create complex AI applications with LlamaIndex', 'date': 'Aug 1, 2024', 'url': 'https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex'}


Workflows keep state

Workflows keep a global state during the execution, and this state can be
shared and propagated to its steps upon request. This shared state is
implemented as a ` Context ` object and can be used by steps to store data in
between iterations but also as an alternative form of communication among
different steps. Let’s see an excerpt from a more complex RAG example as an
example showing how to use the global context (check [ notebook
](https://docs.llamaindex.ai/en/latest/examples/workflow/rag/) for full code):

    
    
    class RAGWorkflow(Workflow):
        @step(pass_context=True)
        async def ingest(self, ctx: Context, ev: StartEvent) -> Optional[StopEvent]:
            dataset_name = ev.get("dataset")
            _, documents = download_llama_dataset(dsname, "./data")
            ctx.data["INDEX"] = VectorStoreIndex.from_documents(documents=documents)
            return StopEvent(result=f"Indexed {len(documents)} documents.")
            
        ...

In this case the ` ingest ` step creates an index, and it wants to make it
available to any other step that might needed it later during workflow
execution. The idiomatic way of doing that in a LlamaIndex workflow is to
declare the step requires an instance of the global context ( `
@step(pass_context=True) ` does the trick) and store the index in the context
itself with a predefined key that other steps might access later.

{'Header_2': ' A workflow primer', 'Header_3': ' Workflows keep state', 'filename': 'introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex.md', 'extension': '.md', 'title': 'Introducing workflows beta: a new way to create complex AI applications with LlamaIndex', 'date': 'Aug 1, 2024', 'url': 'https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex'}


Workflows can be customized

Alongside Workflows, we’ll be releasing a set of predefined workflows so that
the most common use cases can be implemented with a single line of code. Using
these predefined flows, users still might want to just _slightly_ change a
predefined workflow to introduce some custom behavior without having to
rewrite a whole workflow from scratch. Let’s say you want to customize a RAG
workflow and use a custom re-ranking step, all you would need to do is
subclass a hypothetical built-in ` RAGWorkflow ` class and override the `
rerank ` step like this:

    
    
    class MyWorkflow(RAGWorkflow):
        @step(pass_context=True)
        def rerank(
            self, ctx: Context, ev: Union[RetrieverEvent, StartEvent]
        ) -> Optional[QueryResult]:
            # my custom reranking logic here
            
     
    w = MyWorkflow(timeout=60, verbose=True)
    result = await w.run(query="Who is Paul Graham?")

{'Header_2': ' A workflow primer', 'Header_3': ' Workflows can be customized', 'filename': 'introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex.md', 'extension': '.md', 'title': 'Introducing workflows beta: a new way to create complex AI applications with LlamaIndex', 'date': 'Aug 1, 2024', 'url': 'https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex'}


## Semantic Chunking

In [14]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)

In [13]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="models/bge-base-en-v1.5",
    device="cuda:0"
)

In [17]:
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=Settings.embed_model
)

In [18]:
semantic_nodes = splitter.get_nodes_from_documents(loaded_documents, show_progress=True)

Generating embeddings: 100%|██████████| 14/14 [00:00<00:00, 212.39it/s]

Generating embeddings: 100%|██████████| 10/10 [00:00<00:00, 229.24it/s]

Generating embeddings: 100%|██████████| 11/11 [00:00<00:00, 183.99it/s]

Generating embeddings: 100%|██████████| 11/11 [00:00<00:00, 142.25it/s]

Generating embeddings: 100%|██████████| 4/4 [00:00<00:00, 151.67it/s]

Parsing nodes: 100%|██████████| 166/166 [00:58<00:00,  2.86it/s]


In [19]:
for node in semantic_nodes[:10]:
    display(Markdown(node.text))
    print(node.metadata)
    print("=="*50)

Greetings, Llama Lovers!

Welcome to this week’s edition of the LlamaIndex newsletter! We’re excited to
share our latest updates including dynamic features like LlamaIndex Workflows
and retrieval capabilities in LlamaCloud. Check out our in-depth guides,
tutorials, and the upcoming webinars that will help you make the most of these
new developments.

##  **The highlights:**

  1. **LlamaIndex Workflows Launched:** LlamaIndex Workflows, a new event-driven architecture for building multi-agent applications, supports batching, async operations, and streaming. Agents subscribe to and emit events for complex, readable, Pythonic orchestration. [ Blogpost ](https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex) , [ Tweet ](https://x.com/llama_index/status/1819048068798616058) . 
  

{'filename': 'llamaindex-newsletter-2024-08-06.md', 'extension': '.md', 'title': 'LlamaIndex Newsletter 2024-08-06', 'date': 'Aug 6, 2024', 'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-08-06'}


2. **Dynamic Retrieval Feature in LlamaCloud:** A new feature in LlamaCloud now supports dynamic retrieval for QA assistants, enabling both chunk-level and file-level document retrieval based on query similarity to intelligently route queries. [ Blogpost ](https://www.llamaindex.ai/blog/dynamic-retrieval-with-llamacloud) , [ Notebook ](https://github.com/run-llama/llamacloud-demo/blob/main/examples/10k_apple_tesla/demo_file_retrieval.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818337133746360623) . 
  

{'filename': 'llamaindex-newsletter-2024-08-06.md', 'extension': '.md', 'title': 'LlamaIndex Newsletter 2024-08-06', 'date': 'Aug 6, 2024', 'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-08-06'}


3. **LongRAG LlamaPack:** LongRAG is now available as a LlamaPack in LlamaIndex, utilizing larger document chunks and long-context LLMs for more effective synthesis. [ Notebook ](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-longrag/examples/longrag.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818802688274100578) . 

##  **Feature Releases and Enhancements:**

  * We have launched LlamaIndex Workflows, a new event-driven way to build multi-agent applications where each agent acts as a component that subscribes to and emits events, allowing for complex, readable, and Pythonic orchestration with enhanced support for batching, async operations, and streaming. [ Blogpost ](https://www.llamaindex.ai/blog/introducing-workflows-beta-a-new-way-to-create-complex-ai-applications-with-llamaindex) , [ Tweet ](https://x.com/llama_index/status/1819048068798616058) . 
  * We have introduced a new feature in LlamaCloud to improve your QA assistant with our latest capability for dynamic retrieval, allowing both chunk-level and file-level retrieval. This feature enables the retrieval of entire documents based on query similarity, which supports building agents that can intelligently route queries based on their content. [ Blogpost ](https://www.llamaindex.ai/blog/dynamic-retrieval-with-llamacloud) , [ Notebook ](https://github.com/run-llama/llamacloud-demo/blob/main/examples/10k_apple_tesla/demo_file_retrieval.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818337133746360623) . 
  * We have launched LongRAG as a LlamaPack in LlamaIndex. LongRAG simplifies retrieval by using larger document chunks and leveraging long-context LLMs for synthesis. [ Notebook ](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-longrag/examples/longrag.ipynb) , [ Tweet ](https://x.com/llama_index/status/1818802688274100578) . 

**Guides:**

  * [ Guide ](https://docs.llamaindex.ai/en/latest/examples/workflow/react_agent/) to building a ReAct agent from scratch using LlamaIndex workflows. 
  * [ Guide ](https://docs.llamaindex.ai/en/latest/examples/workflow/rag/) to Building an Event-Driven RAG Pipeline with LlamaIndex, featuring distinct event-driven steps for retrieval, reranking, and synthesis, enhanced with graph tracing and async processing. 
  * [ Guide ](https://docs.llamaindex.ai/en/latest/module_guides/observability/#mlflow) to MLflow in LlamaIndex to manage, deploy, and monitor your genAI applications with MLflow's tracking, packaging, evaluation, and tracing capabilities. 

**Tutorials:**

  * [ Pavan Kumar’s ](https://x.com/pavan_mantha1) [ tutorial ](https://blog.gopenai.com/building-smarter-agents-using-llamaindex-agents-and-qdrants-hybrid-search-50c0ecbbfb0d) on Building Smarter Agents using LlamaIndex Agents and Qdrant’s Hybrid Search. 
  * [ Farzad Sunavala’s ](https://www.linkedin.com/in/farzadsunavala) [ tutorial ](https://farzzy.hashnode.dev/rag-observability-and-evaluation-with-azure-ai-search-azure-openai-llamaindex-and-arize-phoenix) on RAG Observability and Evaluation with Azure AI Search, Azure OpenAI, LlamaIndex, and Arize Phoenix. 
  * [ Composio’s ](https://x.com/composiohq) [ tutorial ](https://github.com/ComposioHQ/composio/tree/master/python/examples/pr_agent/pr_agent_llama_index) on building a PR review agent using Composio's GitHub/Slack tools and LlamaIndex agent abstractions. 
  * [ Benito Martin’s ](https://medium.com/@benitomartin) [ tutorial ](https://medium.com/@benitomartin/find-your-code-scaling-a-llamaindex-and-qdrant-application-with-google-kubernetes-engine-2db126f16344) on Scaling a LlamaIndex and Qdrant Application with Google Kubernetes Engine. 
  * [ Chew Loong Nian’s ](https://medium.com/@chewloongnian) [ tutorial ](https://pub.towardsai.net/introducing-llamaextract-beta-transforming-metadata-extraction-for-enhanced-rag-queries-de3d74d34cd7) on Transforming Metadata Extraction for Enhanced RAG Queries using LlamaExtract. 
  * [ Pavan Kumar’s ](https://x.com/pavan_mantha1) [ tutorial ](https://medium.com/@manthapavankumar11/practical-implementation-of-agentic-rag-workflows-with-llama-index-and-qdrant-3b6622cd3124) on Practical Implementation of Agentic RAG Workflows with Llama-Index and Qdrant. 
  * AI21 Labs [ tutorial ](https://www.llamaindex.ai/blog/jamba-instruct-s-256k-context-window-on-llamaindex) on using Jamba-Instruct Model with LlamaIndex. 

**Webinars And Hackathons:**

  * [ Join us ](https://lu.ma/ka5xtyqo) for a webinar on August 8th with [ Dedy Kredo ](https://x.com/DedyKredo) from [ CodiumAI ](https://x.com/CodiumAI) on using RAG with LlamaIndex to help build a code generation solution that’s contextually aware of the right elements of source code. 
  * [ Join us ](https://lu.ma/p13pkknm?tk=SsniSt) on RAG Hack Night at GitHub with [ Weaviate ](https://x.com/weaviate_io) , [ Neosync ](https://x.com/neosynccloud) , [ Arize AI ](https://x.com/arizeai) on August 13th. 



...

{'filename': 'jamba-instruct-s-256k-context-window-on-llamaindex.md', 'extension': '.md', 'title': "Jamba-Instruct's 256k context window on LlamaIndex", 'date': 'Jul 31, 2024', 'url': 'https://www.llamaindex.ai/blog/jamba-instruct-s-256k-context-window-on-llamaindex'}


# Setup the index
    file_list = [os.path.join("data", f) for f in os.listdir("data")]
    
    amzn_10k_docs = SimpleDirectoryReader(input_files=file_list).load_data()
    index = VectorStoreIndex.from_documents(documents=amzn_10k_docs)
    
    # Build a query engine
    default_query_engine = index.as_query_engine(llm)

Let’s enter a query to make sure our RAG system is working.

    
    
    answer = default_query_engine.query("What was the company's revenue in 2021?")
    print(answer.response)
    
    
    The company's revenue in 2021 was $469,822 million.

Great! 

{'filename': 'jamba-instruct-s-256k-context-window-on-llamaindex.md', 'extension': '.md', 'title': "Jamba-Instruct's 256k context window on LlamaIndex", 'date': 'Jul 31, 2024', 'url': 'https://www.llamaindex.ai/blog/jamba-instruct-s-256k-context-window-on-llamaindex'}


It works. Now let’s try a similar query to continue validating.

    
    
    answer = default_query_engine.query("What was the company's revenue in 2023?")
    print(answer.response)
    
    
    The company's revenue in 2023 was not explicitly mentioned in the provided context. However, it is mentioned that the company's operating income increased to $36.9 billion in 2023, compared to $12.2 billion in 2022.

We can see there’s a problem—we know that the answer to our question is most
definitely included in our documents, yet our RAG system is claiming that it
cannot find the answer. That’s because the default amount of retrieved chunks
is rather small (a few chunks). This makes the whole system prone to errors
and failing to capture information that is indeed located in the documents.

However, with Jamba-Instruct, a model which handles a 256K context window
effectively, we can increase the number of retrieved chunks from just a few
(default value) to 100 and vastly improve the entire RAG system.

Let’s build a new query engine on top of our existing index and try the query
that failed before.

    
    
    # Large amount of chunks in the retrieval process
    extended_query_engine = index.as_query_engine(llm,
                                                  similarity_top_k=100)
    
    answer = extended_query_engine.query("What was the company's revenue in 2023?")
    print(answer.response)
    
    
    The company's revenue in 2023 was $574.785 million.

We see that the RAG system, with the help of Jamba-Instruct’s 256K context
window, is now able to produce the accurate answer.

Let’s try one more answer to validate our new RAG system.

    
    
    answer = default_query_engine.query("Was there a stock split in the last five years?")
    print(answer.response)
    
    
    No, there was no stock split in the last five years.
    
    
    answer = extended_query_engine.query("Was there a stock split in the last five years?")
    print(answer.response)
    
    
    Yes, there was a stock split in the last five years. On May 27, 2022, Amazon.com, Inc. effected a 20-for-1 stock split of its common stock.

###  Context is king

Often, the debate is framed as “RAG vs. long context.” We at AI21 Labs believe
that’s the wrong way to look at it. Rather, it’s long context _plus_ RAG. When
paired together in an AI system, a long context model enhances the quality and
accuracy of a RAG system, especially useful in enterprise contexts that
involve lengthy documents or vast databases of information.

Going forward, as RAG systems continue to scale, the number of documents and
lengths of chunks will drastically increase. Only a long context model—whose
context length truly delivers—can handle this amount of text.



{'filename': 'jamba-instruct-s-256k-context-window-on-llamaindex.md', 'extension': '.md', 'title': "Jamba-Instruct's 256k context window on LlamaIndex", 'date': 'Jul 31, 2024', 'url': 'https://www.llamaindex.ai/blog/jamba-instruct-s-256k-context-window-on-llamaindex'}


In [26]:
semantic_docstore = SimpleDocumentStore()

In [27]:
semantic_docstore.add_documents(docs=semantic_nodes)

In [28]:
semantic_docstore.persist("database/llama-blogs-nodes-semantic-parser.json")